# combine json files

In [46]:
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr
import fsspec
import os
import zarr
import dask.array
from tqdm import tqdm
import xarray as xr

In [2]:
fs2 = fsspec.filesystem('')  #local file system to save final jsons to

In [3]:
json_list = fs2.glob("json_files/*.json")

In [10]:
# Set the account name, container name, and file path
account_name = "dasdata"
container_name = "hdf5"
connection_string = os.environ['AZURE_CONSTR_dasdata']

In [47]:
daxs = []
for f in tqdm(json_list):
    storage_options = {
        'fo':f,
        'remote_protocol':'az',
        'remote_options':dict(account_name=account_name)
    }

    z = zarr.open("reference://", storage_options=storage_options)
    da = dask.array.from_zarr(z['Acquisition']['Raw[0]']['RawData'])
    dax = xr.DataArray(da)
    
    daxs.append(dax)

  6%|█████▋                                                                                    | 249/3980 [00:07<01:57, 31.77it/s]


KeyboardInterrupt: 

In [15]:
storage_options = {
    'fo':'json_files/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.json',
    'remote_protocol':'az',
    'remote_options':dict(account_name=account_name)
}

z = zarr.open("reference://", storage_options=storage_options)

In [32]:
da = dask.array.from_zarr(z['Acquisition']['Raw[0]']['RawData'])

In [33]:
da

dask.array<from-zarr, shape=(47500, 12000), dtype=int32, chunksize=(743, 88), chunktype=numpy.ndarray>

In [ ]:
mzz = MultiZarrToZarr(json_list,
    remote_protocol='az',
    remote_options=dict(account_name=account_name),
    concat_dims=['time'],
    identical_dims = ['locus'])

d = mzz.translate()

with fs2.open('ooiDAS_southTx.json', 'wb') as f:
    f.write(ujson.dumps(d).encode())